# Criando comunicação com o Cartesi

## Gerando par de chaves

In [7]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# Generate private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Generate public key
public_key = private_key.public_key()

# Serialize the private key
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)

# Serialize the public key
public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Write the private key to a file
with open('private_key.pem', 'wb') as private_file:
    private_file.write(private_pem)

# Write the public key to a file
with open('public_key.pem', 'wb') as public_file:
    public_file.write(public_pem)

## Assinando mensagem com chave privada

In [8]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization

# Load the private key from the PEM file
with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
    )

# Load the message from the msg.txt file
with open("mensage.txt", "rb") as message_file:
    message = message_file.read()

# Sign the message
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# Write the signature to a file
with open("signature.sig", "wb") as signature_file:
    signature_file.write(signature)

print("Message has been signed and the signature is saved in 'signature.sig'.")

Message has been signed and the signature is saved in 'signature.sig'.


## Gerando JSON para envio ao Cartesi

In [9]:
import json
from cryptography.hazmat.primitives import serialization

# Load the public key from the PEM file
with open("public_key.pem", "rb") as key_file:
    public_key = key_file.read().decode('utf-8')

# Load the raw message from the msg.txt file
with open("mensage.txt", "rb") as message_file:
    raw_message = message_file.read().decode('utf-8')

# Load the signed message from the signature.sig file
with open("signature.sig", "rb") as signature_file:
    signed_message = signature_file.read()

# Convert the signed message to a base64 string for JSON serialization
signed_message_base64 = signed_message.hex()

# Create the dictionary with the public key, raw message, and signed message
data = {
    "public_key": public_key,
    "raw_message": raw_message,
    "signed_message": signed_message_base64,
    "revogation": False  
}

# Write the dictionary to a JSON file
with open("message_data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("JSON file 'message_data.json' has been created with the public key, raw message, and signed message.")


JSON file 'message_data.json' has been created with the public key, raw message, and signed message.
